# Creating Surveys
This notebook contains code for creating variations of questions and combining them into surveys in EDSL. We also show how to seed questions and add stop logic and other rules. We also explore ways or addressing relationships between responses and survey context.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/create_surveys.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionLinearScale, QuestionCheckBox, QuestionYesNo, QuestionMultipleChoice, QuestionFunctional
from edsl.questions.compose_questions import compose_questions
from edsl import Agent, Survey, Scenario

## Question scenarios
Many times we want to re-administer a question with a different parameter or input. We can do this by creating "scenarios" of a question:

In [3]:
items = ["groceries", "clothes", "shoes", "electronics", "home furnishings", "books", "sporting equipment", "wellness items"]

q_shopping = QuestionLinearScale(
    question_name = "q_shopping",
    question_text = "On a scale of 0-10, how much do you typically enjoy shopping for {{item}}?",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0:"Not at all", 10:"Very much"}
)

scenarios = [Scenario({"item":item} )for item in items]

In [4]:
scenarios

[Scenario({'item': 'groceries'}),
 Scenario({'item': 'clothes'}),
 Scenario({'item': 'shoes'}),
 Scenario({'item': 'electronics'}),
 Scenario({'item': 'home furnishings'}),
 Scenario({'item': 'books'}),
 Scenario({'item': 'sporting equipment'}),
 Scenario({'item': 'wellness items'})]

We can use the `by()` method to create an instance of our question for each scenario parameter:

In [5]:
q_shopping.by(scenarios)

Jobs(survey=Survey(questions=[QuestionLinearScale(question_name = 'q_shopping', question_text = 'On a scale of 0-10, how much do you typically enjoy shopping for {{item}}?', question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], short_names_dict = {}, option_labels = {0: 'Not at all', 10: 'Very much'})], name=None), agents=[], models=[], scenarios=[Scenario({'item': 'groceries'}), Scenario({'item': 'clothes'}), Scenario({'item': 'shoes'}), Scenario({'item': 'electronics'}), Scenario({'item': 'home furnishings'}), Scenario({'item': 'books'}), Scenario({'item': 'sporting equipment'}), Scenario({'item': 'wellness items'})])

## Combining questions
We can combine questions to administer them at once with the `add_question()` method:

In [6]:
q1 = QuestionYesNo(
    question_name = "fit",
    question_text = "Do you find it difficult to shop for clothes that fit you?"
) 

q2 = QuestionMultipleChoice(
    question_name = "online",
    question_text = "How often do you shop for clothes online?",
    question_options = ["Never", "Rarely", "Occasionally", "Often"]
)

q1.add_question(q2)

Survey(questions=[QuestionYesNo(question_name = 'fit', question_text = 'Do you find it difficult to shop for clothes that fit you?', question_options = ['Yes', 'No'], short_names_dict = {}), QuestionMultipleChoice(question_name = 'online', question_text = 'How often do you shop for clothes online?', question_options = ['Never', 'Rarely', 'Occasionally', 'Often'], short_names_dict = {})], name=None)

In [7]:
result = q1.add_question(q2).run()

In [8]:
result.select("fit","online").print()

answer.fit,answer.online
No,Occasionally


We can also combine any-sized sets of questions in surveys:

In [9]:
q_factors = QuestionCheckBox(
    question_name = "q_factors",
    question_text = "Which of the following factors are most important to you in making decisions about clothes shopping? Select all that apply.",
    question_options = [
        "Price",
        "Quality",
        "Brand Reputation",
        "Style and Design",
        "Fit and Comfort",
        "Customer Reviews and Recommendations",
        "Ethical and Sustainable Practices",
        "Return Policy",
        "Convenience",
        "Other"
    ]
)

survey = Survey(
    questions = [
        q_shopping,
        q_factors
    ]
)

In [10]:
survey.by(scenarios)

Jobs(survey=Survey(questions=[QuestionLinearScale(question_name = 'q_shopping', question_text = 'On a scale of 0-10, how much do you typically enjoy shopping for {{item}}?', question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], short_names_dict = {}, option_labels = {0: 'Not at all', 10: 'Very much'}), QuestionCheckBox(question_name = 'q_factors', question_text = 'Which of the following factors are most important to you in making decisions about clothes shopping? Select all that apply.', min_selections = None, max_selections = None, question_options = ['Price', 'Quality', 'Brand Reputation', 'Style and Design', 'Fit and Comfort', 'Customer Reviews and Recommendations', 'Ethical and Sustainable Practices', 'Return Policy', 'Convenience', 'Other'], short_names_dict = {})], name=None), agents=[], models=[], scenarios=[Scenario({'item': 'groceries'}), Scenario({'item': 'clothes'}), Scenario({'item': 'shoes'}), Scenario({'item': 'electronics'}), Scenario({'item': 'home furnishings'}), Scenario({'item': 'books'}), Scenario({'item': 'sporting equipment'}), Scenario({'item': 'wellness items'})])

In [11]:
result = survey.by(scenarios).run()

In [12]:
result.select("scenario.*","q_shopping","q_factors").print()

scenario.item,answer.q_shopping,answer.q_factors
shoes,5,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
home furnishings,7,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
wellness items,7,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
books,10,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
sporting equipment,5,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
groceries,7,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
electronics,8,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"
clothes,7,"['Quality', 'Style and Design', 'Fit and Comfort', 'Ethical and Sustainable Practices']"


## Seeding questions
If we want an agent to refer to a prior response in answering a new question, we can seed the new question with a response to a prior question in 2 different ways.

### Method 1: Using the `.add_targeted_memory()` method
Append this method to your survey object with the (2) questions that you want to relate to each other in the following way: `survey.add_targeted_memory(q2, q1)`.

In [13]:
q_eat = QuestionYesNo(
    question_name = "eat",
    question_text = "Do you like to eat {{food}}?"
)

q_sam = QuestionYesNo(
    question_name = "sam",
    question_text = "Are you Sam I Am?"
)

survey = Survey(questions = [q_eat, q_sam])
survey.add_targeted_memory(q_sam, q_eat)

results = survey.by(Scenario({"food":"green eggs and ham"})).run()

We can inspect the `_user_prompt` field of the second question to see that it has been modified to include the first question and response:

In [14]:
results.select("prompt.*").print()

prompt.sam_system_prompt,prompt.eat_user_prompt,prompt.eat_system_prompt,prompt.sam_user_prompt
"{'text': 'You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{}', 'class_name': 'AgentInstruction'}","{'text': 'You are being asked the following question: Do you like to eat green eggs and ham?\nThe options are\n\n0: Yes\n\n1: No\n\nReturn a valid JSON formatted like this, selecting only the number of the option:\n{""answer"": , ""comment"": """"}\nOnly 1 option may be selected.', 'class_name': 'YesNo'}","{'text': 'You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{}', 'class_name': 'AgentInstruction'}","{'text': 'You are being asked the following question: Are you Sam I Am?\nThe options are\n\n0: Yes\n\n1: No\n\nReturn a valid JSON formatted like this, selecting only the number of the option:\n{""answer"": , ""comment"": """"}\nOnly 1 option may be selected.\n Before the question you are now answering, you already answered the following question(s):\n \tQuestion: Do you like to eat green eggs and ham?\n\tAnswer: No', 'class_name': 'YesNo'}"


### Method 2: Using the `compose_questions()` method
You can also use the `compose_questions()` method to include the response to one question in the context of another question:

In [15]:
q_eat = QuestionYesNo(
    question_name = "eat",
    question_text = "Do you like to eat {{food}}?"
)

q_sam = QuestionYesNo(
    question_name = "sam",
    question_text = "You were previously asked: '" + q_eat.question_text 
        + "' You responded: '{{food}}'. Are you Sam I Am?"
)

q_eat_sam = compose_questions(q_eat, q_sam).by(Scenario({"food":"green eggs and ham"}))

In [16]:
q_eat_sam.print()

                               Jobs                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Jobs                                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Questions                                                   ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │ │
│ │ ┃ Question Name ┃ Question Type ┃ Question Text ┃ Options ┃ │ │
│ │ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │ │
│ │ │ eat_sam       │ functional    │ functional    │ None    │ │ │
│ │ └───────────────┴───────────────┴───────────────┴─────────┘ │ │
│ └─────────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────┘

## Survey rules
We can add rules to a survey, such as skip logic and stop logic. Here we at a stop rule to end the survey at the first question if the response is "No":

In [17]:
survey_eat_sam = (q_eat
                 .add_question(q_sam)
                 .add_stop_rule("eat", "eat == 'no'")
)

In [18]:
survey_eat_sam.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓                                      │
│ ┃ Question Name ┃ Question Type ┃ Question Text                ┃ Options ┃                                      │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩                                      │
│ │ eat           │ yes_no        │ Do you like to eat {{food}}? │ Yes, No │                                      │
│ └───────────────┴───────────────┴──────────────────────────────┴─────────┘                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ sam           │ yes_no        │ You were previously asked: 'Do you like to eat {{food}}?' You     │ Yes, No │ │
│ │               │               │ responded: '{{food}}'. Are you Sam I Am?                          │         │ │
│ └───────────────┴───────────────┴───────────────────────────────────────────────────────────────────┴─────────┘ │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

Note that adding the questions creates a Survey, to which we append the stop rule. We can also explicitly create a Survey with the stop rule (these are equivalent):

In [19]:
survey_eat_sam_v2 = Survey([q_eat, q_sam]).add_stop_rule("eat", "eat == 'no'")

In [20]:
survey_eat_sam_v2.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓                                      │
│ ┃ Question Name ┃ Question Type ┃ Question Text                ┃ Options ┃                                      │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩                                      │
│ │ eat           │ yes_no        │ Do you like to eat {{food}}? │ Yes, No │                                      │
│ └───────────────┴───────────────┴──────────────────────────────┴─────────┘                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ sam           │ yes_no        │ You were previously asked: 'Do you like to eat {{food}}?' You     │ Yes, No │ │
│ │               │               │ responded: '{{food}}'. Are you Sam I Am?                          │         │ │
│ └───────────────┴───────────────┴───────────────────────────────────────────────────────────────────┴─────────┘ │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [21]:
from edsl import Agent 

agents = [Agent(traits={"persona":p}) for p in ["You are Sam I am.", "You are John Horton.", "You are a vegan."]]

scenario = Scenario({"food":"green eggs and ham"})

results = survey_eat_sam.by(scenario).by(agents).run()

In [22]:
(results
.select("persona", "scenario.*", "eat", "sam")
.print(pretty_labels={"agent.persona":"Persona", "scenario.food":"Food", "answer.eat":q_eat.question_text, "answer.sam":"Are you Sam I Am?"})
)

Persona,Food,Do you like to eat {{food}}?,Are you Sam I Am?
You are Sam I am.,green eggs and ham,Yes,Yes
You are John Horton.,green eggs and ham,No,No
You are a vegan.,green eggs and ham,No,No


## Survey contexts and administration
We can explore relationships between survey responses and the contexts or conditions under which a survey was "administered". For example, do responses vary depending on whether the questions asked in an anonymous paper or online survey, by a researcher or in a focus group composed of diverse or similar individuals?

This can be explored in `edsl` by adding context about the survey administration to the question text and/or agent traits:

<blockquote>
    <i>Do you exercise every day? (You are being asked this question in a focus group of peers.)</i>
</blockquote>

See an example notebook: <a href="https://deepnote.com/workspace/expected-parrot-c2fa2435-01e3-451d-ba12-9c36b3b87ad9/project/Expected-Parrot-examples-b457490b-fc5d-45e1-82a5-a66e1738a4b9/notebook/survey_admin-e328ca5084e844c599a5b3a99d394731">Survey Administration</a>

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>